In [67]:
from pprint import pprint
from copy import deepcopy

class Aggs():
    def __init__(self, features = [], names=[]):
        if not features:
            raise Exception("Empty function list")
        self.features = features
        self.names = names
        self.data = dict()
        self.n = len(features)
        self.rename = False
        
    def update(self, logs):
        if not type(logs) is list:
            logs = [logs, ]
        map(self.__update_by_one, logs)
        
    def __update_by_one(self, log):
        feature = [f(log) for f in self.features]
        x = self.data

        for i in range(self.n):
            feat = feature[i]

            key = (self.names[i] + ": " + str(feat)) if len(self.names) == self.n and self.rename else str(feat)

            if i < self.n - 1:
                x.setdefault(key, dict())
                x = x[key]
            else:
                x.setdefault(key, [])
                x[key].append(log)
                
            
    def fapp(self, f):
        if not type(f) is list:
            f = [f, ]
        y = deepcopy(self.data)
        for fun in f:
            self.__app(fun, deep = 0, y = y)
        return y
        
    def __app(self, f, deep = 0, y = None):
        for key in y:
            if deep < self.n - 1:
                self.__app(f, deep = deep + 1, y = y[key])
            else:
                y[key] = f(y[key])
                
    def __str__(self):
        pprint(self.data)
        return str(self.data)
    
    def to_list(self, data=None, deep=0):
        if not data:
            data = self.data
        result = []
        if deep < self.n:
            for key in data:
                result += self.to_list(data = data[key], deep = deep + 1)
        else:
            return data
        return result
    
    def merge(self, aggs):
        map(self.update, aggs.to_list())
        
    def reaggr(self, features, names=[]):
        data = self.to_list()
        self.data = dict()
        self.features = features
        self.n = len(features)
        self.names = names
        map(self.update, data)


In [68]:
ag = Aggs([lambda x: x[0],
           lambda x: 'local' in x, 
           lambda x: len(x) > 10, 
           lambda x: len(x) < 9], 
          names=['x[0]', 'local', 'len > 10', 'len(x) < 9'])

ag2 = Aggs([lambda x: x[0],
            lambda x: 'local' in x, 
            lambda x: len(x) > 10, 
            lambda x: len(x) < 9], 
          names=['x[0]', 'local', 'len > 10', 'len(x) < 9'])

In [69]:
ag.update('local')
ag.update('asdjkadsdfsdfsf')
ag.update('asdjkadsdfsdfsf local')
ag.update('al')
ag.update('ald')
ag.update(['awailt', 'wailtmir'])

In [70]:
ag2.update('local2')
ag2.update('asdjkadsdfsdfsf2')
ag2.update('asdjkf local2')
ag2.update('al2')
ag2.update('ald2')
ag2.update(['2awailt', 'wailtmir'])
ag2.update(['[1,2,3]', 'wailtmir', 'wailtmir'])

In [71]:
ag.merge(ag2)

In [72]:
ag.data

{'2': {'False': {'False': {'True': ['2awailt']}}},
 '[': {'False': {'False': {'True': ['[1,2,3]']}}},
 'a': {'False': {'False': {'True': ['al', 'ald', 'awailt', 'al2', 'ald2']},
   'True': {'False': ['asdjkadsdfsdfsf', 'asdjkadsdfsdfsf2']}},
  'True': {'True': {'False': ['asdjkadsdfsdfsf local', 'asdjkf local2']}}},
 'l': {'True': {'False': {'True': ['local', 'local2']}}},
 'w': {'False': {'False': {'True': ['wailtmir',
     'wailtmir',
     'wailtmir',
     'wailtmir']}}}}

In [73]:
ag.fapp(lambda x: map(lambda y: 'a' in y, x))

{'2': {'False': {'False': {'True': [True]}}},
 '[': {'False': {'False': {'True': [False]}}},
 'a': {'False': {'False': {'True': [True, True, True, True, True]},
   'True': {'False': [True, True]}},
  'True': {'True': {'False': [True, True]}}},
 'l': {'True': {'False': {'True': [True, True]}}},
 'w': {'False': {'False': {'True': [True, True, True, True]}}}}

In [74]:
ag.data

{'2': {'False': {'False': {'True': ['2awailt']}}},
 '[': {'False': {'False': {'True': ['[1,2,3]']}}},
 'a': {'False': {'False': {'True': ['al', 'ald', 'awailt', 'al2', 'ald2']},
   'True': {'False': ['asdjkadsdfsdfsf', 'asdjkadsdfsdfsf2']}},
  'True': {'True': {'False': ['asdjkadsdfsdfsf local', 'asdjkf local2']}}},
 'l': {'True': {'False': {'True': ['local', 'local2']}}},
 'w': {'False': {'False': {'True': ['wailtmir',
     'wailtmir',
     'wailtmir',
     'wailtmir']}}}}

In [75]:
ag.fapp([set, lambda x: list(map(lambda y: 'wailt' in y, x))])

{'2': {'False': {'False': {'True': [True]}}},
 '[': {'False': {'False': {'True': [False]}}},
 'a': {'False': {'False': {'True': [True, False, False, False, False]},
   'True': {'False': [False, False]}},
  'True': {'True': {'False': [False, False]}}},
 'l': {'True': {'False': {'True': [False, False]}}},
 'w': {'False': {'False': {'True': [True]}}}}

In [76]:
ag.reaggr([lambda x: len(x)])

In [77]:
print ag

{'13': ['asdjkf local2'],
 '15': ['asdjkadsdfsdfsf'],
 '16': ['asdjkadsdfsdfsf2'],
 '2': ['al'],
 '21': ['asdjkadsdfsdfsf local'],
 '3': ['ald', 'al2'],
 '4': ['ald2'],
 '5': ['local'],
 '6': ['awailt', 'local2'],
 '7': ['[1,2,3]', '2awailt'],
 '8': ['wailtmir', 'wailtmir', 'wailtmir', 'wailtmir']}
{'13': ['asdjkf local2'], '15': ['asdjkadsdfsdfsf'], '21': ['asdjkadsdfsdfsf local'], '16': ['asdjkadsdfsdfsf2'], '3': ['ald', 'al2'], '2': ['al'], '5': ['local'], '4': ['ald2'], '7': ['[1,2,3]', '2awailt'], '6': ['awailt', 'local2'], '8': ['wailtmir', 'wailtmir', 'wailtmir', 'wailtmir']}


In [78]:
from collections import Counter
ag.fapp(Counter)

{'13': Counter({'asdjkf local2': 1}),
 '15': Counter({'asdjkadsdfsdfsf': 1}),
 '16': Counter({'asdjkadsdfsdfsf2': 1}),
 '2': Counter({'al': 1}),
 '21': Counter({'asdjkadsdfsdfsf local': 1}),
 '3': Counter({'al2': 1, 'ald': 1}),
 '4': Counter({'ald2': 1}),
 '5': Counter({'local': 1}),
 '6': Counter({'awailt': 1, 'local2': 1}),
 '7': Counter({'2awailt': 1, '[1,2,3]': 1}),
 '8': Counter({'wailtmir': 4})}

In [79]:
Aggs()

Exception: Empty function list